In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install keras-unet-collection
! pip install segmentation-models

In [ ]:
# required libraries 
import numpy as np 
import pandas as pd
import os
from pathlib import Path
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

from keras_unet_collection import models,losses

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
print(train_df.shape)
train_df.head(4)

In [ ]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)


def build_masks(labels,input_shape, colors=True):
    height, width = input_shape
    if colors:
        mask = np.zeros((height, width, 3))
        for label in labels:
            mask += rle_decode(label, shape=(height,width , 3), color=np.random.rand(3))
    else:
        mask = np.zeros((height, width, 1))
        for label in labels:
            mask += rle_decode(label, shape=(height, width, 1))
    mask = mask.clip(0, 1)
    return mask

def rle2maskResize(rle):
    # CONVERT RLE TO MASK 
    if (len(rle)==0): 
        return np.zeros((256,256) ,dtype=np.uint8)
    
    height= 520
    width = 704
    mask= np.zeros( width*height ,dtype=np.uint8)

    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]-1
    lengths = array[1::2]    
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
    
    return mask.reshape( (height,width), order='F' )[::2,::2]

In [ ]:
sample_filename = '0030fd0e6378'
sample_image_df = train_df[train_df['id'] == sample_filename]
sample_path = f"../input/sartorius-cell-instance-segmentation/train/{sample_image_df['id'].iloc[0]}.png"
sample_img = cv2.imread(sample_path)
sample_rles = sample_image_df['annotation'].values

sample_masks1=build_masks(sample_rles,input_shape=(520, 704), colors=False)
sample_masks2=build_masks(sample_rles,input_shape=(520, 704), colors=True)

fig, axs = plt.subplots(3, figsize=(20, 20))
axs[0].imshow(sample_img)
axs[0].axis('off')

axs[1].imshow(sample_masks1)
axs[1].axis('off')

axs[2].imshow(sample_masks2)
axs[2].axis('off')

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='../input/sartorius-cell-instance-segmentation/train',
                 batch_size=32, dim=(256, 256), n_channels=3,
                 n_classes=3, random_state=2019, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['id'].iloc[ID]
            img_path = f"{self.base_path}/{im_name}.png"
            img = self.__load_grayscale(img_path)
            
            
            # Store samples
            X[i,] = img 

        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['id'].iloc[ID]
            image_df = self.target_df[self.target_df['id'] == im_name]
            
            rles = image_df['annotation'].values
            masks = build_masks(rles,(520,704), colors=False)
            masks = cv2.resize(masks, (256, 256))
            #masks=masks.transpose(1,0)
            masks=np.expand_dims(masks, axis=-1)
            y[i, ] = masks

        return y
    
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        # resize image
        dsize = (256, 256)
        img = cv2.resize(img, dsize)
        
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

In [ ]:
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

In [ ]:
BATCH_SIZE = 16

train_idx, val_idx = train_test_split(
    train_df.index, random_state=2019, test_size=0.2 # mask_count_df
)
train_generator = DataGenerator(
    train_idx, 
    df=train_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=3
)
val_generator = DataGenerator(
    val_idx, 
    df=train_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=3
)

### Defining model

In [ ]:
model1=models.att_unet_2d((256, 256, 3), [64, 128, 256, 512, 1024],n_labels=3,activation='ReLU',output_activation='Sigmoid',weights='imagenet',backbone='EfficientNetB0',batch_norm=True,freeze_backbone=True,pool=False,unpool=False,freeze_batch_norm=True)

### With Segmentation models library

In [ ]:
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

In [ ]:
from segmentation_models import Unet
from segmentation_models.utils import set_trainable


model = Unet('efficientnetb0',input_shape=(256, 256, 3), classes=3, activation='sigmoid',encoder_weights='imagenet')
#inp = Input(shape=(512, 640, 1))
#l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
#out = base_model(l1)
#model = Model(inp, out, name=base_model.name)

model.compile(optimizer='adam', loss=bce_jaccard_loss,metrics=[iou_score,'accuracy']) #bce_dice_loss binary_crossentropy


In [ ]:
model1.compile(optimizer='adam', loss=bce_dice_loss,metrics=[dice_coef,iou_coef,'accuracy'])

In [ ]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

In [ ]:

history = model1.fit(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint],
    use_multiprocessing=False,
    workers=4,
    epochs=10
)

### More Comming Soon..Please Upvote if you like it..